In [55]:
import pandas as pd
import ast
import re

In [56]:
def categorize_hl(row):
    if row <= 7200:
        return 'Low'
    elif 7200 < row <= 18000:
        return 'Medium'
    else:
        return 'High'

In [57]:
def float_or_false(value):
    try:
        if float(value):
            return True
    except ValueError:
        return False

In [58]:
def mod_false(df):
    for i in range(len(df)):
        if df.loc[i,'modifications'] != '':
            df.loc[i,'is_mod'] = 'True'
        else:
            df.loc[i,'is_mod'] = 'False'

In [59]:
def extract_data(text):
    return re.findall(r"'(.*?)'", text)

In [60]:
df = pd.read_csv('PEP_LIFE_original.csv')

In [61]:
df['half_life_data'] = df['half_life_data'].astype(str).str.replace(r'[\[\]():{}]', '', regex=True)
df['modifications'] = df['modifications'].astype(str).str.replace(r'[\'()]', '', regex=True)
df['modifications'] = df['modifications'].str.rstrip(',')

In [62]:
extracted_data = df['half_life_data'].apply(extract_data)
df_expanded = pd.DataFrame(extracted_data.tolist())
#df_expanded[1]=pd.to_numeric(df_expanded[1], errors='coerce')
df['half_life_data'] = df_expanded[1]

In [63]:
df_concat = pd.concat([df, df_expanded], axis=1)

In [64]:
df_concat.drop(columns=[1], inplace=True)

In [65]:
df_concat

,Unnamed: 0,Unnamed: 1,modifications,references,half_life_data,url_sources,0,2,3,4,5,6
0,RRWQWR,(),,['http://www.ncbi.nlm.nih.gov/pubmed/20844765'],1800,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Human serum,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...
1,RRWWRF,(),,['http://www.ncbi.nlm.nih.gov/pubmed/20844765'],1800,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Human serum,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...
2,RRWQWR,"('C-Terminal Amidation',)",C-Terminal Amidation,['http://www.ncbi.nlm.nih.gov/pubmed/20844765'],1800,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Human serum,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...
3,RRWWRF,"('C-Terminal Amidation',)",C-Terminal Amidation,['http://www.ncbi.nlm.nih.gov/pubmed/20844765'],1800,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Human serum,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...
4,RRWQWR,"('N-Terminal',)",N-Terminal,['http://www.ncbi.nlm.nih.gov/pubmed/20844765'],3600,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Human serum,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...
...,...,...,...,...,...,...,...,...,...,...,...,...
1132,FNAPFDVGIKLSGAQYQQHGRAL,"('C-Terminal Amidation',)",C-Terminal Amidation,['http://www.ncbi.nlm.nih.gov/pubmed/17204551'],1320,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Intravenous injection of peptide into Sprague...,vivo_vitro,in vivo,http//crdd.osdd.net/raghava/peplife/display_su...
1133,FNAPFDVGIKLSGVQYQQHSQAL,"('C-Terminal Amidation',)",C-Terminal Amidation,['http://www.ncbi.nlm.nih.gov/pubmed/18602197'],3906,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Mouse plasma,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...
1134,FNAPFDVGIKLSGVQYQQHSQAL,"('C-Terminal Amidation', 'Chemical Modificatio...","C-Terminal Amidation, Chemical Modification: I...",['http://www.ncbi.nlm.nih.gov/pubmed/18602197'],906,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Mouse liver homogenate,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...
1135,ASISGRDTHRLTRTLNCSSIVKEIIGKLPEPELKTDDEGPSLRNKS...,"('Cyclic', 'Chemical Modification: Glycosylati...","Cyclic, Chemical Modification: Glycosylation a...",['http://www.ncbi.nlm.nih.gov/pubmed/8353280'],126,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Injected intravenously in Balb/c mice blood,vivo_vitro,in vivo,http//crdd.osdd.net/raghava/peplife/display_su...


In [66]:
#renombra columnas, cambia a mayusculas la secuencia y pone en mayusculas la primera letra de las modificaciones, ademas de agregar columna is_mod
df_concat = df_concat.rename(columns={'Unnamed: 0': 'sequence', 'half_life_data': 'half_life_seconds', 3: 'experimental_characteristics'})
mod_false(df_concat)
df_concat['sequence'] = df_concat['sequence'].str.upper()
df_concat['modifications'] = df_concat['modifications'].str.title()

In [67]:
#dimensiones de datos duplicados
df_concat[df_concat.duplicated(subset=['sequence', 'experimental_characteristics'], keep=False)].shape

(199, 13)

In [68]:
#dimensiones de datos no duplicados
df_concat[~df_concat.duplicated(subset=['sequence', 'experimental_characteristics'], keep=False)].shape

(938, 13)

In [69]:
#separacion de datos duplicados que tengan la misma secuencia y modificaciones (se examinan las secuencia, si modificaiones y secuencia es igual pero half_life_seconds es diferente se borran todos los duplicados)
df_data_duplicated=(df_concat[df_concat.duplicated(subset=['sequence', 'experimental_characteristics'], keep=False)]).sort_values(by='sequence')
df_data_duplicated

,sequence,Unnamed: 1,modifications,references,half_life_seconds,url_sources,0,2,experimental_characteristics,4,5,6,is_mod
20,AAGIGILTV,"('C-Terminal Amidation',)",C-Terminal Amidation,['http://www.ncbi.nlm.nih.gov/pubmed/10495424'],Stability increased,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,VMM5 cytotoxic T-lymphocyte line incubated in ...,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True
19,AAGIGILTV,"('C-Terminal Amidation', 'N-Terminal')","C-Terminal Amidation, N-Terminal",['http://www.ncbi.nlm.nih.gov/pubmed/10495424'],Stability increased,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,VMM5 cytotoxic T-lymphocyte line incubated in ...,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True
18,AAGIGILTV,"('N-Terminal',)",N-Terminal,['http://www.ncbi.nlm.nih.gov/pubmed/10495424'],Stability increased,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,VMM5 cytotoxic T-lymphocyte line incubated in ...,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True
17,AAGIGILTV,"('Stereochemistry',)",Stereochemistry,['http://www.ncbi.nlm.nih.gov/pubmed/10495424'],Stability increased,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,VMM5 cytotoxic T-lymphocyte line incubated in ...,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True
16,AAGIGILTV,(),,['http://www.ncbi.nlm.nih.gov/pubmed/10495424'],22,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,VMM5 cytotoxic T-lymphocyte line incubated in ...,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,YIGSR,(),,['http://www.ncbi.nlm.nih.gov/pubmed/10544005'],60,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Intravenous injection of labelled peptide in C...,vivo_vitro,in vivo,http//crdd.osdd.net/raghava/peplife/display_su...,False
883,YRGDS,(),,['http://www.ncbi.nlm.nih.gov/pubmed/8218482'],30,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Intravenously injected to C57BL6 mice,vivo_vitro,in vivo,http//crdd.osdd.net/raghava/peplife/display_su...,False
884,YRGDS,"('Chemical Modification: I125 radiolabeling, c...","Chemical Modification: I125 Radiolabeling, Cov...",['http://www.ncbi.nlm.nih.gov/pubmed/8218482'],1800,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Intravenously injected to C57BL6 mice,vivo_vitro,in vivo,http//crdd.osdd.net/raghava/peplife/display_su...,True
1000,YRQSMNNFQGLRSFGCRFGTCTVQKLAHQIYQFTDKDKDNVAPRSK...,"('C-Terminal Amidation', 'Cyclic')","C-Terminal Amidation, Cyclic",['http://www.ncbi.nlm.nih.gov/pubmed/24874704'],37.2,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Male Wistar rats plasma,vivo_vitro,in vivo,http//crdd.osdd.net/raghava/peplife/display_su...,True


In [70]:
#separacion de datos duplicados que tengan la misma secuencia, modificaciones y half_life_seconds (de esto solo se guarda una copia) 
df_data_duplicated_2=(df_concat[df_concat.duplicated(subset=['sequence', 'experimental_characteristics', 'half_life_seconds'], keep=False)]).sort_values(by='sequence')
df_data_duplicated_2

,sequence,Unnamed: 1,modifications,references,half_life_seconds,url_sources,0,2,experimental_characteristics,4,5,6,is_mod
19,AAGIGILTV,"('C-Terminal Amidation', 'N-Terminal')","C-Terminal Amidation, N-Terminal",['http://www.ncbi.nlm.nih.gov/pubmed/10495424'],Stability increased,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,VMM5 cytotoxic T-lymphocyte line incubated in ...,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True
17,AAGIGILTV,"('Stereochemistry',)",Stereochemistry,['http://www.ncbi.nlm.nih.gov/pubmed/10495424'],Stability increased,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,VMM5 cytotoxic T-lymphocyte line incubated in ...,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True
18,AAGIGILTV,"('N-Terminal',)",N-Terminal,['http://www.ncbi.nlm.nih.gov/pubmed/10495424'],Stability increased,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,VMM5 cytotoxic T-lymphocyte line incubated in ...,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True
20,AAGIGILTV,"('C-Terminal Amidation',)",C-Terminal Amidation,['http://www.ncbi.nlm.nih.gov/pubmed/10495424'],Stability increased,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,VMM5 cytotoxic T-lymphocyte line incubated in ...,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True
788,ADSGEGDFLAEGGGVR,(),,['http://www.ncbi.nlm.nih.gov/pubmed/6658706'],30,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Intravenous injection in lateral vein of norma...,vivo_vitro,in vivo,http//crdd.osdd.net/raghava/peplife/display_su...,False
787,ADSGEGDFLAEGGGVR,('Chemical Modification: DAT-FPA was labeled w...,Chemical Modification: Dat-Fpa Was Labeled Wit...,['http://www.ncbi.nlm.nih.gov/pubmed/6658706'],30,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Intravenous injection in lateral vein of norma...,vivo_vitro,in vivo,http//crdd.osdd.net/raghava/peplife/display_su...,True
11,CRRWWRFC,"('C-Terminal Amidation', 'Cyclic')","C-Terminal Amidation, Cyclic",['http://www.ncbi.nlm.nih.gov/pubmed/20844765'],23400,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Human serum,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True
13,CRRWWRFC,"('C-Terminal Amidation', 'N-Terminal', 'Cyclic')","C-Terminal Amidation, N-Terminal, Cyclic",['http://www.ncbi.nlm.nih.gov/pubmed/20844765'],23400,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Human serum,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True
667,ELAGIGILTV,('Chemical Modification: Methylation of carbox...,Chemical Modification: Methylation Of Carboxyl...,['http://www.ncbi.nlm.nih.gov/pubmed/N.A.'],86400,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Serum human,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True
668,ELAGIGILTV,"('C-Terminal Amidation', 'Chemical Modificatio...","C-Terminal Amidation, Chemical Modification: M...",['http://www.ncbi.nlm.nih.gov/pubmed/N.A.'],43200,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Serum human,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True


In [71]:
#separacion de datos no duplicados
df_not_duplicated=(df_concat[~df_concat.duplicated(subset=['sequence', 'experimental_characteristics'], keep=False)]).sort_values(by='sequence')
#se concatenan los datos no duplicados con los datos duplicados pero que se mantiene una copia
df_not_duplicated=pd.concat([df_not_duplicated, df_data_duplicated_2])
df_not_duplicated

,sequence,Unnamed: 1,modifications,references,half_life_seconds,url_sources,0,2,experimental_characteristics,4,5,6,is_mod
588,((POG)5 (PQGPGGPPG) (POG)5)3,"('C-Terminal Amidation', 'N-Terminal')","C-Terminal Amidation, N-Terminal",['http://www.ncbi.nlm.nih.gov/pubmed/18481852'],580,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Not reported,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True
589,((POG)5 (PQGPGSPPG) (POG)5)3,"('C-Terminal Amidation', 'N-Terminal')","C-Terminal Amidation, N-Terminal",['http://www.ncbi.nlm.nih.gov/pubmed/18481852'],910,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Not reported,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True
590,((POG)5 (PVGAAGATG) (POG)5)3,"('C-Terminal Amidation', 'N-Terminal')","C-Terminal Amidation, N-Terminal",['http://www.ncbi.nlm.nih.gov/pubmed/18481852'],345,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Not reported,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True
591,((POG)5 (PVGAASATG) (POG)5)3,"('C-Terminal Amidation', 'N-Terminal')","C-Terminal Amidation, N-Terminal",['http://www.ncbi.nlm.nih.gov/pubmed/18481852'],655,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Not reported,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True
1079,(CBC)KFAP,"('N-Terminal', 'Chemical Modification: Proline...","N-Terminal, Chemical Modification: Proline Rin...",['http://www.ncbi.nlm.nih.gov/pubmed/2991517'],1464,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Dose i/v injected Rat blood,vivo_vitro,in vivo,http//crdd.osdd.net/raghava/peplife/display_su...,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,RRWWRF,(),,['http://www.ncbi.nlm.nih.gov/pubmed/20844765'],1800,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Human serum,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,False
1021,YADAIFT-(CYCLO-D)-SYR-(CYCLO-K)-VLAQLSARKLLQDIMSR,"('C-Terminal Amidation',)",C-Terminal Amidation,['http://www.ncbi.nlm.nih.gov/pubmed/1826667'],14400,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Porcine plasma,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True
1022,YADAIFT-(CYCLO-D)-SYR-(CYCLO-K)-VLAQLSARKLLQDIMSR,"('C-Terminal Amidation', 'Stereochemistry', 'C...","C-Terminal Amidation, Stereochemistry, Chemica...",['http://www.ncbi.nlm.nih.gov/pubmed/1826667'],14400,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Porcine plasma,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True
613,YCGF-NLE-PCW,"('Cyclic', 'Stereochemistry', 'Chemical Modifi...","Cyclic, Stereochemistry, Chemical Modification...",['http://www.ncbi.nlm.nih.gov/pubmed/19762245'],21600,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Rat plasma,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True


In [72]:
#se borran valores nulos y se eliminan los duplicados sobrantes de los que se guarda una copia y se ordenan los datos
df_not_duplicated.dropna(subset=['sequence', 'half_life_seconds'], inplace=True)
df_not_duplicated.drop_duplicates(subset=['sequence', 'experimental_characteristics'], keep='first', inplace=True)
df_not_duplicated.sort_index(inplace=True)

In [73]:
df_not_duplicated

,sequence,Unnamed: 1,modifications,references,half_life_seconds,url_sources,0,2,experimental_characteristics,4,5,6,is_mod
0,RRWQWR,(),,['http://www.ncbi.nlm.nih.gov/pubmed/20844765'],1800,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Human serum,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,False
3,RRWWRF,"('C-Terminal Amidation',)",C-Terminal Amidation,['http://www.ncbi.nlm.nih.gov/pubmed/20844765'],1800,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Human serum,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True
10,CRRWQWRC,"('C-Terminal Amidation', 'Cyclic')","C-Terminal Amidation, Cyclic",['http://www.ncbi.nlm.nih.gov/pubmed/20844765'],5400,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Human serum,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True
11,CRRWWRFC,"('C-Terminal Amidation', 'Cyclic')","C-Terminal Amidation, Cyclic",['http://www.ncbi.nlm.nih.gov/pubmed/20844765'],23400,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Human serum,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True
15,CRRWQWRMKKLGC,"('C-Terminal Amidation', 'Cyclic')","C-Terminal Amidation, Cyclic",['http://www.ncbi.nlm.nih.gov/pubmed/20844765'],6600,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Human serum,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1132,FNAPFDVGIKLSGAQYQQHGRAL,"('C-Terminal Amidation',)",C-Terminal Amidation,['http://www.ncbi.nlm.nih.gov/pubmed/17204551'],1320,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Intravenous injection of peptide into Sprague...,vivo_vitro,in vivo,http//crdd.osdd.net/raghava/peplife/display_su...,True
1133,FNAPFDVGIKLSGVQYQQHSQAL,"('C-Terminal Amidation',)",C-Terminal Amidation,['http://www.ncbi.nlm.nih.gov/pubmed/18602197'],3906,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Mouse plasma,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True
1134,FNAPFDVGIKLSGVQYQQHSQAL,"('C-Terminal Amidation', 'Chemical Modificatio...","C-Terminal Amidation, Chemical Modification: I...",['http://www.ncbi.nlm.nih.gov/pubmed/18602197'],906,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Mouse liver homogenate,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True
1135,ASISGRDTHRLTRTLNCSSIVKEIIGKLPEPELKTDDEGPSLRNKS...,"('Cyclic', 'Chemical Modification: Glycosylati...","Cyclic, Chemical Modification: Glycosylation A...",['http://www.ncbi.nlm.nih.gov/pubmed/8353280'],126,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Injected intravenously in Balb/c mice blood,vivo_vitro,in vivo,http//crdd.osdd.net/raghava/peplife/display_su...,True


In [74]:
df_not_duplicated['hl_is_float'] = df_not_duplicated['half_life_seconds'].apply(float_or_false)
df_float=df_not_duplicated[df_not_duplicated['hl_is_float']==True]
df_object=df_not_duplicated[df_not_duplicated['hl_is_float']==False]
df_float.reset_index(drop=True, inplace=True)
df_object.reset_index(drop=True, inplace=True)

In [75]:
df_float

,sequence,Unnamed: 1,modifications,references,half_life_seconds,url_sources,0,2,experimental_characteristics,4,5,6,is_mod,hl_is_float
0,RRWQWR,(),,['http://www.ncbi.nlm.nih.gov/pubmed/20844765'],1800,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Human serum,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,False,True
1,RRWWRF,"('C-Terminal Amidation',)",C-Terminal Amidation,['http://www.ncbi.nlm.nih.gov/pubmed/20844765'],1800,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Human serum,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True,True
2,CRRWQWRC,"('C-Terminal Amidation', 'Cyclic')","C-Terminal Amidation, Cyclic",['http://www.ncbi.nlm.nih.gov/pubmed/20844765'],5400,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Human serum,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True,True
3,CRRWWRFC,"('C-Terminal Amidation', 'Cyclic')","C-Terminal Amidation, Cyclic",['http://www.ncbi.nlm.nih.gov/pubmed/20844765'],23400,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Human serum,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True,True
4,CRRWQWRMKKLGC,"('C-Terminal Amidation', 'Cyclic')","C-Terminal Amidation, Cyclic",['http://www.ncbi.nlm.nih.gov/pubmed/20844765'],6600,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Human serum,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941,FNAPFDVGIKLSGAQYQQHGRAL,"('C-Terminal Amidation',)",C-Terminal Amidation,['http://www.ncbi.nlm.nih.gov/pubmed/17204551'],1320,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Intravenous injection of peptide into Sprague...,vivo_vitro,in vivo,http//crdd.osdd.net/raghava/peplife/display_su...,True,True
942,FNAPFDVGIKLSGVQYQQHSQAL,"('C-Terminal Amidation',)",C-Terminal Amidation,['http://www.ncbi.nlm.nih.gov/pubmed/18602197'],3906,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Mouse plasma,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True,True
943,FNAPFDVGIKLSGVQYQQHSQAL,"('C-Terminal Amidation', 'Chemical Modificatio...","C-Terminal Amidation, Chemical Modification: I...",['http://www.ncbi.nlm.nih.gov/pubmed/18602197'],906,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Mouse liver homogenate,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True,True
944,ASISGRDTHRLTRTLNCSSIVKEIIGKLPEPELKTDDEGPSLRNKS...,"('Cyclic', 'Chemical Modification: Glycosylati...","Cyclic, Chemical Modification: Glycosylation A...",['http://www.ncbi.nlm.nih.gov/pubmed/8353280'],126,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Injected intravenously in Balb/c mice blood,vivo_vitro,in vivo,http//crdd.osdd.net/raghava/peplife/display_su...,True,True


In [76]:
df_object

,sequence,Unnamed: 1,modifications,references,half_life_seconds,url_sources,0,2,experimental_characteristics,4,5,6,is_mod,hl_is_float
0,AAGIGILTV,"('C-Terminal Amidation', 'N-Terminal')","C-Terminal Amidation, N-Terminal",['http://www.ncbi.nlm.nih.gov/pubmed/10495424'],Stability increased,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,VMM5 cytotoxic T-lymphocyte line incubated in ...,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True,False
1,RPPGFSPFR,(),,['http://www.ncbi.nlm.nih.gov/pubmed/10871321'],Not measured as peptide degraded rapidly30 app...,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Human blood sample,vivo_vitro,in vivo,http//crdd.osdd.net/raghava/peplife/display_su...,False,False
2,HSEGTFTSDYSKYLDSRRAQDFVQWLMNTKRNRNNIA,(),,['http://www.ncbi.nlm.nih.gov/pubmed/19602537'],Stable,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Mouse plasma Subcutaneous injection,vivo_vitro,in vivo,http//crdd.osdd.net/raghava/peplife/display_su...,False,False


In [77]:
#Por si se quieren quitar las modificaciones
df_filtered = df_float[df_float['is_mod'] == 'False']
df_filtered = df_filtered.drop(columns=['is_mod', 'modifications'])
df_filtered_ob = df_object[df_object['is_mod'] == 'False']
df_filtered_ob = df_filtered_ob.drop(columns=['is_mod', 'modifications'])

In [78]:
df_float=df_float.filter(['sequence', 'half_life_seconds', 'is_mod', 'modifications'])
df_object=df_object.filter(['sequence', 'half_life_seconds', 'is_mod', 'modifications', 'experimental_characteristics'])

In [79]:
df_float['half_life_seconds'] = df_float['half_life_seconds'].astype(float)

In [80]:
df_float['hl_category'] = df_float['half_life_seconds'].apply(categorize_hl)

In [81]:
#df_filtered.to_csv('../datasets/PEP_LIFE.csv', index=False)
#df_filtered_ob.to_csv('../datasets/PEP_LIFE.csv', index=False)
df_float.to_csv('PEP_LIFE.csv', index=False)
df_object.to_csv('PEP_LIFE_object.csv', index=False)